In [6]:
import requests
import boto3 
import time
import random
import json

from datetime import datetime



def get_user():
    r = requests.get('https://randomuser.me/api/?exc=login')
    user = "" 
    
    # AWS Lambda function will fail, as it will look for keys. need to safeguard.
    # safeguard added to lambda function to return failures on empty dicts 
    # Initialize empty user
    
    try:
        user= json.dumps(r.json()['results'][0])
    except:
        print(r.text)
        print(r.status_code, datetime.now())
        
    return str(user)


def put_firehose(firehose_name, user):
    firehose_name = firehose_name
    firehose_client  = boto3.client('firehose', region_name="eu-west-1")
    
    # boto3 client handles uuid for partition key
    firehose_client.put_record(DeliveryStreamName=firehose_name,
                                           Record={'Data': user})
    
def get_user_put_data(firehose_name, m):
    
    firehose_name = firehose_name
    firehose_client  = boto3.client('firehose', region_name="eu-west-1")

    i = 0
    while i < m:
        user = get_user()
        put_firehose("stream-marketing-users", user)
        
        # add a jitter 
        time.sleep(random.uniform(1,5))   
        i = i+1
    
    return 



In [7]:
import multiprocessing
from datetime import datetime


firehose_name = "stream-marketing-users"
now = datetime.now()
 
print("start test =", now)

jobs = []

# launch num_workers, to make num_requests each

num_workers = 1
num_requests = 10

for i in range(num_workers):
    p = multiprocessing.Process(target=get_user_put_data, args=(firehose_name, num_requests))
    jobs.append(p)
    p.start()
        
print("end test =", now)

start test = 2020-11-19 08:55:27.128565
end test = 2020-11-19 08:55:27.128565
<html>
<head>
<title>Uh oh, something bad happened</title>
<script async src='/cdn-cgi/bm/cv/2172558837/api.js'></script></head>
<body align='center'>
<h1 align='center'>Uh oh, something bad happened</h1>
<a class="twitter-timeline" data-width="450" data-height="700" data-theme="light" href="https://twitter.com/randomapi">Tweets by randomapi</a> <script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>
<script type="text/javascript">(function(){window['__CF$cv$params']={r:'5f48b6617b420792',m:'f24dd62a18b14ec2517566b4c64b0b5445e11710-1605776128-1800-ARJ25nOlBSv1LvAoL+0KXm2tkHxAk96tYt1ofTMYwszHwYOExCkRhoL4/OXTm7870XWRSNkDYaokAIZug0ib5eJgN3Zuja0HoVrNr17Cmelgxt4mxY3y9Kd5slD9/MulTA==',s:[0x6ced688581,0xa7dfdf61b7],}})();</script></body>
</html>

503 2020-11-19 08:55:28.333060
